In [1]:
try:
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow as tf
assert tf.__version__.startswith('1')

tf.enable_eager_execution()

import os
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [3]:
!unzip '/content/train.zip'

Archive:  /content/train.zip
   creating: data/bajra/
  inflating: data/bajra/1511340347-4701.jpg  
  inflating: data/bajra/1583912022_Jowar_big_2.jpg  
  inflating: data/bajra/2aa98bcad23f92a695505239de73d941.jpg  
  inflating: data/bajra/49ba5b2c0360ba0658fdacd4e94b6ec4.jpg  
  inflating: data/bajra/5000-hybrid-pioneer-86m11-bajra-seeds-pearl-millet-seeds-1-5kg-original-imaft7xedxesp9aa.jpeg  
  inflating: data/bajra/5000-trimurti-tmbh-2652-bajra-seeds-pearl-millet-seeds-1-5kg-original-imaft8gvykbfke5n.jpeg  
  inflating: data/bajra/531f61_42baae386ec44ea7bdda77804c5feefemv2_d_2992_2000_s_2.jpg  
  inflating: data/bajra/853476-bajra-072919.jpg  
  inflating: data/bajra/8b9f17943f53dab4aa225b9235552ab2.jpg  
  inflating: data/bajra/bajra.jpg    
  inflating: data/bajra/bajra0.jpg   
  inflating: data/bajra/bajra21.jpg  
  inflating: data/bajra/Bajra-Cultivation.jpg  
  inflating: data/bajra/bajra-millet-DFYWMR.jpg  
  inflating: data/bajra/bajra-millet-pennisetum-glaucum-field-rajguru

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/content/data',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    '/content/data',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 794 images belonging to 10 classes.
Found 190 images belonging to 10 classes.


In [5]:
image_batch, label_batch = next(val_generator)
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 10))

In [6]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('crop_labels.txt', 'w') as f:
  f.write(labels)

{'bajra': 0, 'corn': 1, 'cotton': 2, 'jute': 3, 'rice': 4, 'sugarcane': 5, 'sunflower': 6, 'tea': 7, 'tobacco': 8, 'wheat': 9}


In [7]:
!cat crop_labels.txt

bajra
corn
cotton
jute
rice
sugarcane
sunflower
tea
tobacco
wheat

In [8]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')
base_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [9]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(units=5, activation='softmax')
])